In [4]:
import csv
import random
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [24]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0"
}
recipe_url = "https://www.10000recipe.com/recipe/list.html?order=reco&page="
url = "https://www.10000recipe.com/recipe/"
food_data = []
recipe_data = []

try:
    # --- 1. 음식 목록 긁어 오기 ---
    for i in tqdm(range(50), desc="페이지 목록 수집 중"):
        response = requests.get(recipe_url + str(i + 1), headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        food_items = soup.select('li.common_sp_list_li')

        for item in food_items:
            id_tag = item.select_one('a.common_sp_link')
            title_tag = item.select_one('div.common_sp_caption_tit')
            author_tag = item.select_one('div.common_sp_caption_rv_name')
            view_tag = item.select_one('div.common_sp_caption_rv span.common_sp_caption_buyer')
            img_tag = item.select_one('img[src*="/cache/recipe"]')

            id_val = id_tag['href'].replace('/recipe/', '') if id_tag and 'href' in id_tag.attrs else 'None'
            title = title_tag.get_text(strip=True) if title_tag else 'None'
            author = author_tag.get_text(strip=True) if author_tag else 'None'
            view = view_tag.get_text().replace('조회수', '').strip() if view_tag else 'None'
            img = img_tag['src'] if img_tag and 'src' in img_tag.attrs else 'None'

            food_data.append({
                'ID': id_val,
                'Title': title,
                'Author': author,
                'View': view,
                'Image': img
            })
        time.sleep(random.uniform(0.5, 1))

    print(f"\n총 {len(food_data)}개의 레시피 리스트 확보.")

    # --- 2. 레시피 상세 정보 긁어 오기 ---
    for food in tqdm(food_data, desc="레시피 상세 정보 수집 중"):
        try:
            response = requests.get(url + str(food['ID']), headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # 링크
            recipe_link = url + str(food['ID'])

            # 기본 정보 추출
            serv_tag = soup.select_one('span.view2_summary_info1')
            prep_tag = soup.select_one('span.view2_summary_info2')
            diff_tag = soup.select_one('span.view2_summary_info3')

            difficulty = diff_tag.get_text(strip=True) if diff_tag else 'None'

            if difficulty not in ['아무나', '초급']:
                continue

            serving = serv_tag.get_text(strip=True) if serv_tag else 'None'
            prep_time = prep_tag.get_text(strip=True) if prep_tag else 'None'

            main_thumb_id = soup.select_one('#main_thumbs')
            main_thumb = main_thumb_id['src'] if main_thumb_id else 'None'

            summary_in_id = soup.select_one('#recipeIntro')
            summary_in = summary_in_id.get_text(strip=True) if summary_in_id else 'None'

            # 재료 추출
            ingred_items = soup.select('div.ready_ingre3 ul')
            ingredient_text = 'None'
            condiment_text = 'None'

            if not ingred_items:
                continue
            else:
                for item in ingred_items:
                    title_tag = item.select_one('b.ready_ingre3_tt')
                    group_title = title_tag.get_text(strip=True) if title_tag else 'None'
                    ingredients = item.select('li')
                    ingredient_list = []
                    for ingredient_item in ingredients:
                        name_tag = ingredient_item.select_one('div.ingre_list_name a') or ingredient_item.select_one('div.ingre_list_name')
                        ea_tag = ingredient_item.select_one('span.ingre_list_ea')
                        i_name = name_tag.get_text(strip=True) if name_tag else ''
                        i_ea = ea_tag.get_text(strip=True) if ea_tag else ''
                        if i_name:
                            ingredient_list.append(f'{i_name}-{i_ea}')
                    joined_ingredients = ','.join(ingredient_list)
                    if '재료' in group_title or '레시피' in group_title:
                        ingredient_text = joined_ingredients
                    elif '양념' in group_title:
                        condiment_text = joined_ingredients

            # --- [수정됨] 조리 순서(Steps) 이미지 추출 로직 개선 ---
            step_divs = soup.select('div.view_step_cont')
            step_list = []

            skip_recipe = False
            for i, step_div in enumerate(step_divs):
                desc_tag = step_div.select_one('div.media-body')
                desc = desc_tag.get_text(strip=True) if desc_tag else "None"

                # 수정 포인트: src 속성에 '/recipe/'가 포함된 이미지를 찾음 (구조 무관)
                img_tag = step_div.select_one('img[src*="/recipe/"]')

                # 만약 위 방식으로 안 잡히면 일반 img 태그 시도 (아이콘 등 제외 필요 시 필터링)
                if not img_tag:
                    img_tag = step_div.select_one('img')

                # 조리순서에 이미지 없으면 스킵
                img_src = img_tag['src'] if img_tag else "None"
                if img_src == 'None':
                    skip_recipe = True
                    break

                step_str = f"{i+1}단계||{desc}||{img_src}"
                step_list.append(step_str)

            if skip_recipe:
                continue
            all_steps_text = "\n".join(step_list)

            recipe_data.append({
                'Recipe_Link': recipe_link,
                'Title': food['Title'],
                'Serving': serving,
                'Preparation_Time': prep_time,
                'Difficulty': difficulty,
                'Main_Thumbnail': main_thumb,
                'Summary_In': summary_in,
                'Ingredient': ingredient_text,
                'Condiment': condiment_text,
                'Steps': all_steps_text
            })

            time.sleep(random.uniform(0.5, 1))

        except Exception as e:
            print(f"ID {food['ID']} 처리 중 에러 발생: {e}")
            continue

    print("\n--- 추출 결과 확인 (상위 1개) ---")
    if recipe_data:
        print(recipe_data[0])

except Exception as e:
    print(f"전체 프로세스 중 치명적 오류: {e}")

# --- 3. CSV 저장 ---
fieldnames = ['Recipe_Link', 'Title', 'Serving', 'Preparation_Time', 'Difficulty', 'Main_Thumbnail', 'Summary_In', 'Ingredient', 'Condiment', 'Steps']

filename = 'recipe_data.csv'
with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(recipe_data)

print(f"'{filename}' 파일 저장 완료.")

페이지 목록 수집 중: 100%|██████████| 50/50 [00:58<00:00,  1.18s/it]



총 2000개의 레시피 리스트 확보.


레시피 상세 정보 수집 중:  15%|█▍        | 297/2000 [06:45<29:28,  1.04s/it]

ID 7027794 처리 중 에러 발생: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


레시피 상세 정보 수집 중: 100%|██████████| 2000/2000 [43:20<00:00,  1.30s/it]


--- 추출 결과 확인 (상위 1개) ---
{'Recipe_Link': 'https://www.10000recipe.com/recipe/7069205', 'Title': '양파 요리, 굴소스 양파계란볶음 레시피', 'Serving': '2인분', 'Preparation_Time': '20분 이내', 'Difficulty': '아무나', 'Main_Thumbnail': 'https://recipe1.ezmember.co.kr/cache/recipe/2026/01/03/1c9d9e86727d9e3fa5ae33c744fcceac1.jpg', 'Summary_In': '양파 요리, 굴소스 양파계란볶음 레시피 입니다.', 'Ingredient': '계란-3알,양파-1개,대파-1대,식용유-2T,굴소스-1T,깨소금-1T,참기름-1T', 'Condiment': 'None', 'Steps': '1단계||대파는 원하는 크기로 썰어주시고, 양파는 얇게 슬라이스 해주세요.||https://recipe1.ezmember.co.kr/cache/recipe/2026/01/03/4bc4bf520fb4be6178c0f1c86ea17a151.jpg\n2단계||계란 3알을 깨서 잘 풀어 준비합니다.||https://recipe1.ezmember.co.kr/cache/recipe/2026/01/03/9b89fc22ca45d5d7210ab3655e397ade1.jpg\n3단계||달군 프라이팬에 식용유 2T 가량 둘러주세요.||https://recipe1.ezmember.co.kr/cache/recipe/2026/01/03/1d6ba1819ed98e5bde672237b5693cb11.jpg\n4단계||썰어놓은 대파와 양파를 모두 넣어 볶아 줍니다.||https://recipe1.ezmember.co.kr/cache/recipe/2026/01/03/e95af9f903e7e82075d47ed6dc9347101.jpg\n5단계||중불에서 약 2~3분 정도 볶아 준다음 프라이팬의 한쪽으로 양파와 대파를

In [55]:
df = pd.read_csv('recipe_data.csv')
df

,Recipe_Link,Title,Serving,Preparation_Time,Difficulty,Main_Thumbnail,Summary_In,Ingredient,Condiment,Steps
0,https://www.10000recipe.com/recipe/7069205,"양파 요리, 굴소스 양파계란볶음 레시피",2인분,20분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,"양파 요리, 굴소스 양파계란볶음 레시피 입니다.","계란-3알,양파-1개,대파-1대,식용유-2T,굴소스-1T,깨소금-1T,참기름-1T",NaN,"1단계||대파는 원하는 크기로 썰어주시고, 양파는 얇게 슬라이스 해주세요.||htt..."
1,https://www.10000recipe.com/recipe/7065887,굴소스 팽이버섯 계란 덮밥 만들기 한 그릇 레시피,1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,바쁜 날에도 10분 만에 완성되는 굴소스 팽이버섯 계란 덮밥 만들기 한 그릇 레시피...,"밥-1공기,팽이버섯-1봉지,계란-2개,대파-","식용유-1T,간장-1T,굴소스-1T,물-3T,참기름-1T",1단계||재료 준비||https://recipe1.ezmember.co.kr/cac...
2,https://www.10000recipe.com/recipe/7009944,오이고추된장무침 (사계절 먹기좋은 밑반찬 고추무침 5분 레시피),4인분,5분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,오이고추로 만든 밥반찬 밑반찬입니다 불 없이 초간단하게 만들 수 있는 레시피입니다,오이고추-7개,"된장-1스푼,고추장-0.5스푼,참기름-1스푼,올리고당-0.5스푼,통깨-1스푼","1단계||볼에 된장 양념을 모두 넣고 골고루 섞어준다믹싱볼,숟가락||https://..."
3,https://www.10000recipe.com/recipe/6993517,새송이버섯버터굴소스볶음 간단반찬,2인분,5분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,미니새송이버섯으로 만든 간단반찬입니다,"미니새송이버섯-1팩,버터-20g,굴소스-1T,진간장-0.5T,통깨-약간",NaN,1단계||새송이버섯은 길게 편 썰어 주세요||https://recipe1.ezmem...
4,https://www.10000recipe.com/recipe/7025833,매운빨간콩나물무침 레시피 매일반찬,4인분,15분 이내,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,큰아이가 오랜만에 요청한 매운빨간콩나물무침입니다.,"콩나물-300g,대파-1/2대,고추가루-1.5T,맛소금-1/3T,멸치액젓-1T,국간...",NaN,"1단계||콩나물 300G을 잘 세척해 끓는 물에 넣고 3~4분정도 삶아줍니다.볼,채..."
...,...,...,...,...,...,...,...,...,...,...
1805,https://www.10000recipe.com/recipe/6929581,"한그릇요리, 시금치 갈릭 새우 볶음밥",1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,한그릇 요리로 즐겨도 되고반찬으로 먹어도 좋아요 ^-^!!새우에는 단백질이 풍부하고...,"시금치-1줌,새우-1/2컵,밥-1그릇,식용유-3큰술","진간장-1큰술,다진마늘-1큰술,통깨-1큰술,소금-약간","1단계||새우 껍질이 있는경우엔껍질을 먼저 제거해줍니다.생새우, 칵테일새우, 새우살..."
1806,https://www.10000recipe.com/recipe/6947570,새우 오이 볶음 다이어트 도시락 반찬,2인분,30분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,담백함이 가득해자극적이지 않지만 자꾸 끌리는 맛꼬들꼬들 쫄깃쫄깃 식감까지 좋아,"오이-1.5개,칵테일 새우-2줌,대파-약간,청양고추-1개","소금-,다진 마늘-1T,굴 소스-1t,후춧가루-,맛술-,깨-",1단계||새우에 밑간을 해 줍니다.칵테일 새우는 해동 후 물에 헹궈 준 후 꼬리를 ...
1807,https://www.10000recipe.com/recipe/7039703,콩나물국,4인분,15분 이내,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,찬바람이 불고 겨울이 오면서 우리집에는 국이 없으면 밥을 먹지 못하는 남편 덕분에 ...,"콩나물-1봉지,다진마늘-1스푼,대파-적당량,국간장-1스푼,소금-약간,코인육수-2알",NaN,1단계||콩나물국을 소개합니다.||https://recipe1.ezmember.co...
1808,https://www.10000recipe.com/recipe/6879004,한우로 끓이는 얼큰한 경상도식 소고기국,2인분,30분 이내,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,한우로 끓인 얼큰하고 시원한 경상도식 소고기국해장용으로도 좋아요,"한우 양지-300g,무-1/4개,콩나물-1줌,대파-1대,국간장-4T,진간장-1T,참...",NaN,1단계||소고기는 한우 양지로 준비해주세요.찬물에 30분정도 담가 핏물을 충분히 빼...


In [68]:
df['Ingredient'] = df['Ingredient'].str.replace('​', '')
df['Condiment'] = df['Condiment'].str.replace('​', '')
df.to_csv('recipe_data_v2.csv', index=False, encoding='utf-8-sig')

In [58]:
df_ingredient = df['Ingredient'].str.split(',').explode().str.split('-').str[0]
df_ingredient.dropna(inplace=True)
df_ingredient = df_ingredient.str.replace(' ', '').str.replace('​', '')
df_ingredient.drop_duplicates(inplace=True)
df_ingredient.sort_values(inplace=True)
df_ingredient.reset_index(drop=True, inplace=True)
df_ingredient.to_csv('ingredients.csv', index=False, encoding='utf-8-sig')
df_ingredient

0       (우유)식빵
1         2배식초
2       8인치또띠아
3       가는고춧가루
4          가래떡
         ...  
1425       흰설탕
1426    흰설탕굴릴것
1427       흰쌀밥
1428       흰양파
1429        흰자
Name: Ingredient, Length: 1430, dtype: object

In [59]:
df_condiment = df['Condiment'].str.split(',').explode().str.split('-').str[0]
df_condiment.dropna(inplace=True)
df_condiment = df_condiment.str.replace(' ', '').str.replace('​', '')
df_condiment.drop_duplicates(inplace=True)
df_condiment.sort_values(inplace=True)
df_condiment.reset_index(drop=True, inplace=True)
df_condiment.to_csv('condiments.csv', index=False, encoding='utf-8-sig')
df_condiment

0           2배식초
1           가는소금
2            간마늘
3             간장
4         간장2숟가락
         ...    
326         후춧가루
327      후춧가루탈탈탈
328    휘핑크림or요거트
329          흑설탕
330       히말라얀소금
Name: Condiment, Length: 331, dtype: object

In [66]:
import pandas as pd
import re

# 1. 파일 불러오기 (condiment.csv가 원본이라고 가정)
df_condiments = pd.read_csv('condiments.csv')

# 2. [1단계] 텍스트 청소 함수 (Regex)
# 숫자, 단위, 괄호, '또는' 등 불필요한 문자를 기계적으로 제거
def clean_text_regex(text):
    if not isinstance(text, str):
        return text

    # 1. '또는', 'or' 기준 앞부분만 선택
    text = re.split(r'또는|or|혹은|/', text)[0]

    # 2. 괄호 제거 (예: 핫소스(타바스코) -> 핫소스)
    text = re.sub(r'\(.*?\)', '', text)

    # 3. 수량/단위 제거 (매우 강력함)
    # 숫자 뒤에 단위가 붙은 패턴 삭제 (1T, 300g, 1개 등)
    text = re.sub(r'\d+(숟가락|스푼|큰술|작은술|티스푼|컵|g|ml|L|kg|개|알|장|모|봉|T|Ts|t|can|캔).*', '', text)

    # 4. 남은 숫자 제거 (맨 뒤에 붙은 숫자 등)
    text = re.sub(r'\d+$', '', text)

    # 5. 잡다한 수식어 제거
    garbage = ['약간', '취향껏', '적당량', '듬뿍', '조금', '탈탈탈', '솔솔', '톡톡', '선택', '생략가능', '곱게간것', '수북하게']
    for word in garbage:
        text = text.replace(word, '')

    return text.strip()

# 1단계 적용
df['Cleaned'] = df_condiments.apply(clean_text_regex)

# 3. [2단계] 핵심 재료 추출 및 통일 (Extraction Map)
# 여기가 가장 중요합니다. 발견된 모든 이음동의어를 여기에 등록했습니다.
extraction_map = {
    # [고추/매운맛]
    '고춧가루': '고춧가루', '고추가루': '고춧가루', '땡초가루': '고춧가루', '레드페퍼': '고춧가루',
    '고추장': '고추장', '초고추장': '고추장',
    '청양고추': '청양고추', '청양초': '청양고추', '땡초': '청양고추', '매운고추': '청양고추',
    '홍고추': '홍고추', '붉은고추': '홍고추',
    '고추기름': '고추기름', '라유': '고추기름',
    '페퍼론치노': '페퍼론치노', '스리라차': '스리라차', '핫소스': '핫소스',

    # [마늘/파/양파/생강]
    '다진마늘': '다진마늘', '간마늘': '다진마늘', '마늘': '마늘', '통마늘': '마늘',
    '양파': '양파', '다진양파': '양파',
    '대파': '대파', '파': '대파', '쪽파': '대파', '실파': '대파', '다진파': '대파', '송송썬파': '대파',
    '생강': '생강', '다진생강': '생강', '생강가루': '생강',

    # [소스류]
    '굴소스': '굴소스',
    '돈가스소스': '돈가스소스', '돈까스소스': '돈가스소스',
    '토마토소스': '토마토소스', '스파게티소스': '토마토소스',
    '칠리소스': '칠리소스', '스위트칠리': '칠리소스',
    '머스타드': '머스타드', '머스터드': '머스타드', '홀그레인': '머스타드', '겨자': '겨자', '연겨자': '겨자',
    '마요네즈': '마요네즈', '마요네즈': '마요네즈',
    '케찹': '케찹', '케첩': '케찹', '케챱': '케찹',
    '된장': '된장', '쌈장': '쌈장', '춘장': '춘장',
    '와사비': '와사비', '생와사비': '와사비', '고추냉이': '와사비',
    '우스터소스': '우스터소스', '데리야끼': '데리야끼소스',
    '짜장가루': '짜장가루', '카레': '카레', '카레가루': '카레',

    # [기름류] - 식용유로 대통합
    '올리브유': '식용유', '올리브오일': '식용유', '오일': '식용유',
    '식용유': '식용유', '식용류': '식용유', '기름': '식용유', '포도씨유': '식용유', '해바라기유': '식용유', '현미유': '식용유', '카놀라유': '식용유',
    '참기름': '참기름', '들기름': '들기름',
    '버터': '버터', '무염버터': '버터', '가염버터': '버터', '마가린': '버터',

    # [간장/액젓]
    '진간장': '간장', '국간장': '간장', '양조간장': '간장', '맛간장': '간장', '간장': '간장', '조림간장': '간장',
    '쯔유': '쯔유', '가쓰오부시장국': '쯔유',
    '액젓': '액젓', '멸치액젓': '액젓', '까나리액젓': '액젓', '참치액': '액젓', '피쉬소스': '액젓',
    '새우젓': '새우젓', '새우젓갈': '새우젓',

    # [가루/조미료]
    '소금': '소금', '맛소금': '소금', '천일염': '소금', '꽃소금': '소금', '허브솔트': '소금', '구운소금': '소금', '함초소금': '소금',
    '설탕': '설탕', '황설탕': '설탕', '백설탕': '설탕', '흑설탕': '설탕', '슈가파우더': '설탕', '스테비아': '설탕', '알룰로스': '설탕',
    '올리고당': '올리고당', '물엿': '올리고당', '요리당': '올리고당', '조청': '올리고당', '꿀': '꿀',
    '조미료': '조미료', '다시다': '조미료', '미원': '조미료', '치킨스톡': '치킨스톡', '연두': '조미료', '라면스프': '조미료',
    '후추': '후추', '후춧가루': '후추', '통후추': '후추', '블랙페이퍼': '후추', '순후추': '후추',
    '참깨': '참깨', '통깨': '참깨', '깨소금': '참깨', '깨': '참깨', '검은깨': '참깨', '흑임자': '참깨', '깻가루': '참깨',
    '식초': '식초', '2배식초': '식초', '사과식초': '식초', '현미식초': '식초', '레몬즙': '식초', '레몬쥬스': '식초', '발사믹': '식초',
    '맛술': '맛술', '미림': '맛술', '소주': '맛술', '청주': '맛술', '와인': '맛술',
    '매실청': '매실청', '매실액': '매실청', '매실원액': '매실청', '매실': '매실청',

    # [유제품/잼/기타]
    '치즈': '치즈', '모짜렐라': '치즈', '체다치즈': '치즈', '파마산': '치즈',
    '우유': '우유', '생크림': '생크림', '휘핑크림': '생크림',
    '요거트': '요거트', '요구르트': '요거트',
    '딸기잼': '딸기잼', '딸기쨈': '딸기잼', '땅콩잼': '땅콩버터',
    '전분': '전분', '전분가루': '전분', '녹말가루': '전분', '감자전분': '전분',
    '밀가루': '밀가루', '부침가루': '부침가루', '튀김가루': '튀김가루', '빵가루': '빵가루', '찹쌀가루': '찹쌀가루',
    '계란': '계란', '달걀': '계란', '계란물': '계란',
    '물': '물', '육수': '육수', '다시마': '다시마', '멸치': '멸치', '국멸치': '멸치',
    '김': '김', '김가루': '김',
    '파슬리': '파슬리', '바질': '바질', '로즈마리': '로즈마리', '계피가루': '계피가루', '시나몬': '계피가루'
}

# 맵핑을 위해 키 길이 순으로 정렬 (긴 단어 우선 매칭)
sorted_keys = sorted(extraction_map.keys(), key=len, reverse=True)

def extract_standard_name(text):
    if not isinstance(text, str):
        return text

    # 1. 매핑된 단어가 포함되어 있는지 검사 (긴 단어 우선)
    for key in sorted_keys:
        if key in text:
            return extraction_map[key]

    # 2. 매핑 안 된 단어는 그대로 반환하되, 너무 길거나 이상하면 버리기 위해 일단 반환
    return text

# 2단계 적용
df['Final_Condiment'] = df['Cleaned'].apply(extract_standard_name)

# 4. [3단계] 최종 정제 및 저장
# 빈 값 제거, 중복 제거, 가나다순 정렬
final_df = df[['Final_Condiment']].dropna()
# 공백만 있는 데이터 제거
final_df = final_df[final_df['Final_Condiment'].str.strip() != '']
# 중복 제거
final_df = final_df.drop_duplicates().sort_values(by='Final_Condiment')
final_df.columns = ['Condiment']

# "꾸밈재료", "은박컵" 같은 쓰레기 데이터 필터링 (길이로 1차 필터링)
# 보통 조미료는 2~5글자 내외임. 10글자 넘어가면 정제 실패한 찌꺼기일 확률 높음
final_df = final_df[final_df['Condiment'].str.len() < 10]

# 결과 출력
print(f"최종 조미료 개수: {len(final_df)}개")
print(final_df.head(50))

# 저장
final_df.to_csv('condiments_cleaned.csv', index=False, encoding='utf-8-sig')
print("\n'condiments_cleaned.csv' 파일로 저장되었습니다.")

최종 조미료 개수: 91개
    Condiment
3          간장
9          감자
14         겨자
16         계란
18       계피가루
22       고추기름
24        고추장
19       고춧가루
36        굴소스
44          김
53       꾸밈재료
54          꿀
59        다시마
2        다진마늘
66       다진부추
68      다진소고기
70     다진오이피클
76         달래
62         대파
81     데친브로콜리
82      돈가스소스
84         된장
87        들기름
93        딸기잼
95       땅콩버터
99         레몬
103      로즈마리
104        마늘
106      마요네즈
111        맛술
114       매실청
122      머스타드
35         멸치
32          물
146       밀가루
148       바나나
149        바질
152         배
153        배즙
156        버터
168      부침가루
172       빵가루
174       새우젓
67         생강
189        생수
328       생크림
155        설탕
1          소금
202    송송썬청고추
205       순두부

'condiments_cleaned.csv' 파일로 저장되었습니다.
